In [1]:
import numpy as np
from sklearn.cluster import KMeans
import  open3d as o3d
import numpy as np
import open3d as o3d
import os
import re
file_num = 0

# Example data (replace with actual point cloud and initial labels)
# load points and labels
label_to_color = {
        0: (0.0, 0.0, 0.0),
        1: (0.0, 1.0, 0.0),
        2: (1.0, 0.0, 0.0),
    }
for file_num in range(8):
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = [n for n in names if pattern_ply.match(n)]
    for i in range(1, len(names_ply) + 1):
        pc_path = f'../data/{file_num}/{i}.ply'  # replace with your point cloud file
        label_path = f'../data/{file_num}/{i}_labels.npy'  # replace with your label file
        pcd = o3d.io.read_point_cloud(pc_path)
        points = np.asarray(pcd.points)        # shape (N, 3)
        labels = np.load(label_path).reshape(-1).astype(np.int32)# 1 = safe, 0 = unsafe, -1 = unlabeled

        # Apply K-Means clustering to all points
        K = 25 # for example, 15 clusters (tune this as needed)
        kmeans = KMeans(n_clusters=K, random_state=42)
        cluster_ids = kmeans.fit_predict(points)

        # Determine majority label in each cluster
        cluster_label = {}  # map cluster_id -> assigned label
        for cid in range(K):
            idx = np.where(cluster_ids == cid)[0]               # indices of points in this cluster
            labeled_idx = idx[labels[idx] != -1]                # indices of labeled points in cluster
            if labeled_idx.size == 0:
                continue  # no label info in this cluster; skip assigning
            # Count safe vs unsafe among known labels in cluster
            safe_count = np.sum(labels[labeled_idx] == 1)
            unsafe_count = np.sum(labels[labeled_idx] == 2)
            # Decide label if one class dominates
            total_labeled = safe_count + unsafe_count
            if total_labeled > 0:
                majority_label = 1 if safe_count > unsafe_count else 2
                frac = max(safe_count, unsafe_count) / total_labeled
                # print(f"Cluster {cid} - Safe: {safe_count}, Unsafe: {unsafe_count}, Fraction: {frac:.2f}")
                if frac >= 0.70:  # e.g., require at least 70% agreement
                    cluster_label[cid] = majority_label
        # Assign labels to unlabeled points based on cluster majority
        new_labels = labels.copy()
        for cid, lbl in cluster_label.items():
            # print(f"Cluster {cid} majority label: {lbl}")
            # print(f"Cluster {cid} majority label: {lbl}")
            idx = np.where(cluster_ids == cid)[0]
            # Label only those that were unlabeled
            unl_idx = idx[new_labels[idx] == 0]
        #     new_labels[unl_idx] = lbl
        # # save the new labels
        # print("Total number oif pints", len(new_labels)-len(labels))
        # print(f"number of zero label before the point cloud: {np.sum(new_labels == 0)- np.sum(labels == 0)}")
        np.save(f'../data/{file_num}/{i}_labels.npy', new_labels)
        print("done ", file_num)
    # print(f"number of zero label after the point cloud: {np.sum(new_labels == 0)}")
    # # 'new_labels' now contains expanded labels for points that met the criteria
    # print("unique labels after clustering:", np.unique(new_labels))
    # # visualize the results
    # pcd.colors = o3d.utility.Vector3dVector(np.array([label_to_color[l] for l in new_labels], dtype=np.float64))
    # o3d.visualization.draw_geometries([pcd])



ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import numpy as np
import open3d as o3d
import os
import re
from sklearn.neighbors import NearestNeighbors
from collections import Counter

label_to_color = {
    0: (0.0, 0.0, 0.0),
    1: (0.0, 1.0, 0.0),
    2: (1.0, 0.0, 0.0),
}

k = 50  # Number of neighbors
threshold = 0.4  # 70% majority required to assign label

for file_num in range(8):
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = [n for n in names if pattern_ply.match(n)]
    
    for i in range(1, len(names_ply) + 1):
        pc_path = f'../data/{file_num}/{i}.ply'
        label_path = f'../data/{file_num}/{i}_labels.npy'
        
        pcd = o3d.io.read_point_cloud(pc_path)
        points = np.asarray(pcd.points)
        labels = np.load(label_path).reshape(-1).astype(np.int32)

        labeled_idx = np.where((labels == 1) | (labels == 2))[0]
        unlabeled_idx = np.where(labels == 0)[0]

        if labeled_idx.size == 0 or unlabeled_idx.size == 0:
            continue

        labeled_points = points[labeled_idx]
        unlabeled_points = points[unlabeled_idx]
        labeled_labels = labels[labeled_idx]

        nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(labeled_points)
        distances, indices = nbrs.kneighbors(unlabeled_points)

        new_labels = labels.copy()

        for j, neighbor_idxs in enumerate(indices):
            neighbor_labels = labeled_labels[neighbor_idxs]
            label_counts = Counter(neighbor_labels)
            most_common_label, count = label_counts.most_common(1)[0]
            if count / k >= threshold:
                new_labels[unlabeled_idx[j]] = most_common_label

        # np.save(f'../data/{file_num}/{i}_labels.npy', new_labels)
        # print("Done:", file_num, i)
        # Map labels to colors (make sure the label_to_color dictionary is defined)
        colors = np.array([label_to_color.get(lbl, (0.5, 0.5, 0.5)) for lbl in new_labels], dtype=np.float64)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd], window_name=f"Point Cloud {file_num}-{i}")



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
import numpy as np
import open3d as o3d
import CSF
import os, re
# ---- your existing setup ----
# file_num = 0
# i = 5
for file_num in range(8):
    file_num =7
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = [n for n in names if pattern_ply.match(n)]
    for i in range(1, len(names_ply) + 1):
        pcd = o3d.io.read_point_cloud(f'../data/{file_num}/{i}.ply')
        xyz = np.asarray(pcd.points)

        # 2) Configure the CSF filter
        csf = CSF.CSF()
        csf.params.bSloopSmooth    = False    # smooth out small terrain noise
        csf.params.cloth_resolution = 0.03    # size of cloth mesh cells (m)
        # (optional) tweak these too:
        csf.params.rigidness    = 1
        csf.params.iterations   = 500
        csf.params.class_threshold = 0.1

        # 3) Run the filter
        csf.setPointCloud(xyz)
        ground_idx     = CSF.VecInt()       # will hold indices of ground points
        non_ground_idx = CSF.VecInt()       # will hold indices of obstacles
        csf.do_filtering(ground_idx, non_ground_idx)

        # 4) Extract and color
        ground_pts   = xyz[np.array(list(ground_idx),     dtype=int)]
        obstacle_pts = xyz[np.array(list(non_ground_idx), dtype=int)]
        ground_pc     = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(ground_pts))
        obstacle_pc   = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(obstacle_pts))

        ground_pc.paint_uniform_color([0.0, 1.0, 0.0])   # gray = ground
        obstacle_pc.paint_uniform_color([1.0, 0.0, 0.0]) # red  = obstacles

        # 5) Visualize
        o3d.visualization.draw_geometries([ground_pc, obstacle_pc])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import open3d as o3d
import CSF

BASE_DIR = '../data'
SLOPE_THRESHOLD_DEG = 10.0   # ramp if normal deviates more than this from vertical

for file_num in range(8):
    file_num =7
    folder = os.path.join(BASE_DIR, str(file_num))
    if not os.path.isdir(folder):
        continue

    ply_files = sorted(f for f in os.listdir(folder) if f.lower().endswith('.ply'))
    for ply_name in ply_files:
        # 1) load
        ply_path = os.path.join(folder, ply_name)
        pcd = o3d.io.read_point_cloud(ply_path)
        xyz = np.asarray(pcd.points)

        # 2) estimate normals once on full cloud
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
        normals = np.asarray(pcd.normals)  # shape (N,3)

        # 3) run CSF
        csf = CSF.CSF()
        csf.params.bSloopSmooth     = True
        csf.params.cloth_resolution = 0.1
        csf.params.rigidness        = 5
        csf.params.iterations       = 500
        csf.params.class_threshold  = 0.5

        csf.setPointCloud(xyz)
        ground_idx     = CSF.VecInt()
        non_ground_idx = CSF.VecInt()
        csf.do_filtering(ground_idx, non_ground_idx)

        # convert to Python lists
        ground_idx_list     = np.array(list(ground_idx), dtype=int)
        non_ground_idx_list = np.array(list(non_ground_idx), dtype=int)

        # 4) detect ramps in the ground set
        g_normals = normals[ground_idx_list]                   # normals for CSF‐ground
        # angle from vertical = arccos(|n·[0,0,1]|)
        angles = np.degrees(np.arccos(np.clip(np.abs(g_normals[:,2]), -1.0, 1.0)))
        ramp_mask = angles > SLOPE_THRESHOLD_DEG

        ramp_idx_global = ground_idx_list[ramp_mask]           # these are the ramps
        flat_idx_global = ground_idx_list[~ramp_mask]          # truly flat ground

        # 5) assemble final indices
        final_ground_idx   = flat_idx_global
        final_obstacle_idx = np.concatenate([non_ground_idx_list, ramp_idx_global])

        # 6) extract point clouds
        ground_pc   = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz[final_ground_idx]))
        obstacle_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz[final_obstacle_idx]))

        # color: green=flat ground, blue=ramps, red=other obstacles
        ground_pc.paint_uniform_color([0.0, 1.0, 0.0])
        # optionally visualize ramps separately:
        ramp_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz[ramp_idx_global]))
        ramp_pc.paint_uniform_color([0.0, 0.0, 1.0])
        obstacle_pc.paint_uniform_color([1.0, 0.0, 0.0])
        o3d.visualization.draw_geometries(
            [ground_pc, ramp_pc, obstacle_pc],
            window_name=f"Segmentation: {ply_name}",
            width=800,
            height=600,
            mesh_show_back_face=True
        )
        # visua;oze 
        # # 7) write or visualize
        # o3d.io.write_point_cloud(os.path.join(folder, ply_name.replace('.ply','_flat.ply')),   ground_pc)
        # o3d.io.write_point_cloud(os.path.join(folder, ply_name.replace('.ply','_ramp.ply')),   ramp_pc)
        # o3d.io.write_point_cloud(os.path.join(folder, ply_name.replace('.ply','_obst.ply')),   obstacle_pc)

        # print(f"{ply_name}: {len(flat_idx_global)} flat, {len(ramp_idx_global)} ramp, "
        #       f"{len(non_ground_idx_list)} other obstacles")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


KeyboardInterrupt: 

In [ ]:
import numpy as np
import open3d as o3d
import CSF
import os, re
from sklearn.neighbors import NearestNeighbors
from collections import Counter

for file_num in range(8):
    file_num =  0 # <- You overwrite this, so the loop always uses 7
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = [n for n in names if pattern_ply.match(n)]

    for i in range(1, len(names_ply) + 1):
        # Load point cloud
        pcd = o3d.io.read_point_cloud(f'../data/{file_num}/{i}.ply')
        xyz = np.asarray(pcd.points)
        
        # Load labels
        label_path = f'../data/{file_num}/{i}_labels.npy'
        labels = np.load(label_path).reshape(-1).astype(np.int32)

        # --- Run CSF to classify ground vs obstacle ---
        csf = CSF.CSF()
        csf.params.bSloopSmooth = False
        csf.params.cloth_resolution = 0.03
        csf.params.rigidness = 1
        csf.params.iterations = 500
        csf.params.class_threshold = 0.1

        csf.setPointCloud(xyz)
        ground_idx = CSF.VecInt()
        non_ground_idx = CSF.VecInt()
        csf.do_filtering(ground_idx, non_ground_idx)

        # Convert CSF indices to numpy arrays
        ground_idx = np.array(list(ground_idx), dtype=int)
        obstacle_idx = np.array(list(non_ground_idx), dtype=int)

        # Assign ground/obstacle labels (1 = safe, 2 = unsafe)
        csf_labels = np.zeros_like(labels)
        csf_labels[ground_idx] = 1
        csf_labels[obstacle_idx] = 2

        # --- Apply KNN to propagate CSF labels to originally unlabeled points ---
        k = 10
        threshold = 0.7

        labeled_idx = np.where(csf_labels != 0)[0]
        unlabeled_idx = np.where(labels == 0)[0]

        if labeled_idx.size > 0 and unlabeled_idx.size > 0:
            knn = NearestNeighbors(n_neighbors=k).fit(xyz[labeled_idx])
            distances, neighbor_indices = knn.kneighbors(xyz[unlabeled_idx])

            for j, neighbors in enumerate(neighbor_indices):
                neighbor_labels = csf_labels[labeled_idx[neighbors]]
                most_common, count = Counter(neighbor_labels).most_common(1)[0]
                if count / k >= threshold:
                    labels[unlabeled_idx[j]] = most_common

        # Save updated labels
        np.save(label_path, labels)

        # --- Visualization ---
        label_to_color = {
            0: (0.5, 0.5, 0.5),  # unlabeled
            1: (0.0, 1.0, 0.0),  # safe
            2: (1.0, 0.0, 0.0),  # unsafe
        }

        colors = np.array([label_to_color.get(l, (0.5, 0.5, 0.5)) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd], window_name=f"File {file_num}, Frame {i}")


clothes labelling

In [ ]:
import open3d as o3d
import numpy as np
import os, re, cv2
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import CSF
from tqdm import tqdm
output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)



for file_num in tqdm(range(1,2)):  # Just 0 for now
    file_output_dir = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(file_output_dir, exist_ok=True)
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = sorted([n for n in names if pattern_ply.match(n)], key=lambda x: int(x.split('.')[0]))
    frame_id = 0
    for i in range(1, len(names_ply) + 1):
        # Load point cloud
        pcd = o3d.io.read_point_cloud(f'../data/{file_num}/{i}.ply')
        xyz = np.asarray(pcd.points)

        label_path = f'../data/{file_num}/{i}_labels.npy'
        labels = np.load(label_path).reshape(-1).astype(np.int32)

        # --- CSF ---
        csf = CSF.CSF()
        csf.params.bSloopSmooth = False
        csf.params.cloth_resolution = 0.03
        csf.params.rigidness = 1
        csf.params.iterations = 500
        csf.params.class_threshold = 0.1

        csf.setPointCloud(xyz)
        ground_idx = CSF.VecInt()
        non_ground_idx = CSF.VecInt()
        csf.do_filtering(ground_idx, non_ground_idx)

        ground_idx = np.array(list(ground_idx), dtype=int)
        obstacle_idx = np.array(list(non_ground_idx), dtype=int)

        csf_labels = np.zeros_like(labels)
        csf_labels[ground_idx] = 1
        csf_labels[obstacle_idx] = 2

        # --- KNN ---
        k = 10
        threshold = 0.7
        labeled_idx = np.where(csf_labels != 0)[0]
        unlabeled_idx = np.where(labels == 0)[0]

        if labeled_idx.size > 0 and unlabeled_idx.size > 0:
            knn = NearestNeighbors(n_neighbors=k).fit(xyz[labeled_idx])
            _, neighbor_indices = knn.kneighbors(xyz[unlabeled_idx])
            for j, neighbors in enumerate(neighbor_indices):
                neighbor_labels = csf_labels[labeled_idx[neighbors]]
                most_common, count = Counter(neighbor_labels).most_common(1)[0]
                if count / k >= threshold:
                    labels[unlabeled_idx[j]] = most_common

        np.save(label_path, labels)

        # # --- Assign colors and render to image ---
        # label_to_color = {
        #     0: (0.5, 0.5, 0.5),
        #     1: (0.0, 1.0, 0.0),
        #     2: (1.0, 0.0, 0.0),
        # }
        # colors = np.array([label_to_color.get(l, (0.5, 0.5, 0.5)) for l in labels])
        # pcd.colors = o3d.utility.Vector3dVector(colors)

        # vis = o3d.visualization.Visualizer()
        # vis.create_window(visible=False)
        # vis.add_geometry(pcd)
        # vis.update_geometry(pcd)
        # vis.poll_events()
        # vis.update_renderer()
        # vis.capture_screen_image(f"{output_frame_dir}/{file_num}/frame_{frame_id:04d}.png")
        # vis.destroy_window()
        # frame_id += 1


100%|██████████| 1/1 [00:36<00:00, 36.07s/it]


In [ ]:
import cv2
import os

frame_dir = 'rendered_frames'
file_num = 1
frame_path = os.path.join(frame_dir, str(file_num))
frame_files = sorted([f for f in os.listdir(frame_path) if f.endswith('.png')])

if not frame_files:
    raise ValueError("No frames found to create the video.")

# Read first image to determine size
first_frame = cv2.imread(os.path.join(frame_path, frame_files[0]))
height, width, _ = first_frame.shape
out = cv2.VideoWriter('output_1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))

# Write all frames to video
for frame_file in frame_files:
    img = cv2.imread(os.path.join(frame_path, frame_file))
    if img is not None:
        out.write(img)

out.release()


# Label points

In [3]:
!pip install cloth-simulation-filter

In [1]:
import os
import re
import json
import open3d as o3d
import numpy as np
import CSF
import cv2
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from collections import Counter

def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz,   2*qx*qy-2*qz*qw,     2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,     1-2*qx*qx-2*qz*qz,   2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,     2*qy*qz+2*qx*qw,     1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

def compute_labels_for_frame(track_path, frame_idx, threshold=1.2):
    """
    Compute the 0/1/2 labels for frame `frame_idx` in folder `track_path`
    using the same proximity + collision logic from your first script.
    """
    plyf   = os.path.join(track_path, f"{frame_idx}.ply")
    car0f  = os.path.join(track_path, f"{frame_idx}_car_state.json")
    coll0f = os.path.join(track_path, f"{frame_idx}_collision_info.json")

    if not (os.path.exists(plyf) and os.path.exists(car0f) and os.path.exists(coll0f)):
        raise FileNotFoundError(f"Missing base files for frame {frame_idx} in {track_path}")

    # load point cloud & initial car pose
    cloud = o3d.io.read_point_cloud(plyf)
    pts   = np.asarray(cloud.points)
    car0  = json.load(open(car0f))
    pos0, R0 = get_pose(car0)
    world_pts = (R0 @ pts.T).T + pos0

    ever_close     = np.zeros(len(world_pts), dtype=bool)
    ever_collision = np.zeros(len(world_pts), dtype=bool)

    # look at all future frames j = 1,2,3,...
    j = 1
    while True:
        carjf  = os.path.join(track_path, f"{j}_car_state.json")
        colljf = os.path.join(track_path, f"{j}_collision_info.json")
        if not (os.path.exists(carjf) and os.path.exists(colljf)):
            break

        carj  = json.load(open(carjf))
        collj = json.load(open(colljf))

        # proximity
        posj, _ = get_pose(carj)
        d_car = np.linalg.norm(world_pts - posj, axis=1)
        ever_close |= (d_car < (threshold - 0.2))

        # collision
        if collj['has_collided']:
            cp = collj['position']
            pos_col = np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
            d_col = np.linalg.norm(world_pts - pos_col, axis=1)
            ever_collision |= (d_col < (threshold + 0.2))

        j += 1

    # final labels: 0=uncertain, 1=safe, 2=risk
    labels = np.zeros(len(world_pts), dtype=np.uint8)
    labels[ ever_close & ~ever_collision ] = 1
    labels[ ever_collision ] = 2

    return labels

# ——————— main processing loop ———————
output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)

data_root = "../data"
threshold_check = 1.2   # or whatever you prefer
k = 10                  # for KNN
knn_thresh = 0.7        # for label propagation

for file_num in tqdm(range(0, 8), desc="Tracks"):  # change upper bound as needed
    track_folder = os.path.join(data_root, str(file_num))
    names = os.listdir(track_folder)
    ply_files = sorted(
        [n for n in names if re.match(r"^\d+\.ply$", n)],
        key=lambda x: int(x.split(".")[0])
    )

    for ply_name in tqdm(ply_files, desc=f"Frames in track {file_num}"):
        idx = int(ply_name.split(".")[0])
        # load point cloud
        pcd = o3d.io.read_point_cloud(os.path.join(track_folder, ply_name))
        xyz = np.asarray(pcd.points)

        # --- compute labels on the fly instead of np.load(...) ---
        labels = compute_labels_for_frame(track_folder, idx, threshold=threshold_check)

        # --- CSF ground/obstacle split ---
        csf = CSF.CSF()
        csf.params.bSloopSmooth = False
        csf.params.cloth_resolution = 0.03
        csf.params.rigidness = 1
        csf.params.iterations = 500
        csf.params.class_threshold = 0.1

        csf.setPointCloud(xyz)
        ground_idx = CSF.VecInt()
        non_ground_idx = CSF.VecInt()
        csf.do_filtering(ground_idx, non_ground_idx)

        ground_idx = np.array(list(ground_idx), dtype=int)
        obstacle_idx = np.array(list(non_ground_idx), dtype=int)

        csf_labels = np.zeros_like(labels, dtype=np.int32)
        csf_labels[ground_idx]  = 1
        csf_labels[obstacle_idx] = 2

        # --- KNN propagation to fill in any 0s ---
        labeled_idx   = np.where(csf_labels != 0)[0]
        unlabeled_idx = np.where(labels == 0)[0]

        if labeled_idx.size and unlabeled_idx.size:
            knn = NearestNeighbors(n_neighbors=k).fit(xyz[labeled_idx])
            _, nbrs = knn.kneighbors(xyz[unlabeled_idx])
            for j, neigh in enumerate(nbrs):
                most_common, cnt = Counter(csf_labels[labeled_idx[neigh]]).most_common(1)[0]
                if cnt / k >= knn_thresh:
                    labels[unlabeled_idx[j]] = most_common

        out_npy = os.path.join(track_folder, f"{idx}_labels.npy")
        # print(out_npy)
        np.save(out_npy, labels)


        # colors = np.zeros_like(xyz)
        # colors[labels == 0] = [0.5, 0.5, 0.5]  # gray
        # colors[labels == 1] = [0.0, 1.0, 0.0]  # green
        # colors[labels == 2] = [1.0, 0.0, 0.0]  # red
        # pcd.colors = o3d.utility.Vector3dVector(colors)

        # o3d.visualization.draw_geometries(
        #     [pcd],
        #     window_name=f"Track {file_num} – Frame {idx}",
        #     width=800, height=600
        # )


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Tracks:  75%|███████▌  | 6/8 [30:22<10:07, 303.74s/it]


KeyboardInterrupt: 

### Visualize and captyure as IMage


In [ ]:
import open3d as o3d
import numpy as np
import os
import json
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,     2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,     1 - 2*qx**2 - 2*qy**2]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    rot = quat2rot(o)
    return pos, rot

label_to_color = {
    0: (0.5, 0.5, 0.5),
    1: (0.0, 1.0, 0.0),
    2: (1.0, 0.0, 0.0),
}

output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)

# --- build car mesh ---
car_mesh = o3d.geometry.TriangleMesh.create_box(width=0.8, height=0.4, depth=0.3)
car_mesh.compute_vertex_normals()
car_mesh.paint_uniform_color([0.0, 0.0, 0.0])        # red car
center = car_mesh.get_center()
car_mesh.translate(-center)

# --- build a front arrow ---
arrow = o3d.geometry.TriangleMesh.create_arrow(
    cylinder_radius=0.03,    # shaft thickness
    cone_radius=0.06,        # tip thickness
    cylinder_height=0.2,     # shaft length
    cone_height=0.1          # tip length
)
arrow.compute_vertex_normals()
arrow.paint_uniform_color([1.0, 0.0, 0.0])   # make it bright red

# default arrow points along +Z, so rotate it to point along +X

R_y_neg90 = R.from_euler('y', -np.pi/2).as_matrix()
arrow.rotate(R_y_neg90, center=(0,0,0))
arrow.translate([0.2, 0, 0.5])

for file_num in tqdm(range(8), desc="Tracks"):
    file_output_dir = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(file_output_dir, exist_ok=True)

    data_dir = f'../data/{file_num}'
    ply_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.ply')],
                       key=lambda x: int(x[:-4]))

    # one window per track
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=1280, height=720, visible=False)

    ctr = vis.get_view_control()
    
    for ply_name in tqdm(ply_files, desc=f"Frame in track {file_num}"):
        idx = ply_name[:-4]

        # --- load
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, ply_name))
        labels = np.load(os.path.join(data_dir, f"{idx}_labels.npy")).astype(int).reshape(-1)
        with open(os.path.join(data_dir, f"{idx}_car_state.json")) as f:
            car_state = json.load(f)

        # --- colorize
        cols = np.array([label_to_color.get(l, label_to_color[0]) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(cols)
        mirror_x = np.eye(4)
        mirror_x[0, 0] = -1.0
        pcd.transform(mirror_x)

        # --- get pose & flatten to yaw only
        pos, rot_full = get_pose(car_state)
        pos[0] *= -1  # mirror X
        rot_full = np.diag([-1, 1, 1]) @ rot_full  # mirror orientation

        yaw = np.arctan2(rot_full[1, 0], rot_full[0, 0])
        rot_flat = R.from_euler('z', yaw, degrees=False).as_matrix()

        # --- chase‐cam in flattened frame
        offset_local = np.array([-7, 0, -5.0])
        lookat = car_mesh.get_center() + np.array([0.0, 0.0, 0.95])
        cam_pos = car_mesh.get_center() + ( offset_local)

        # --- render
        vis.clear_geometries()
        vis.add_geometry(pcd)
        vis.add_geometry(car_mesh)  
        vis.add_geometry(arrow)
        ctr.set_lookat( lookat )
        front = (lookat - cam_pos)
        front /= np.linalg.norm(front)
        ctr.set_front( front.tolist() )
        ctr.set_up([0, 0, 1])
        ctr.set_zoom(0.4)

        vis.poll_events()
        vis.update_renderer()
        # vis.run()
        # time.sleep(0.1)
        img_path = os.path.join(file_output_dir, f"{idx}.png")
        vis.capture_screen_image(img_path)

    vis.destroy_window()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Tracks:  25%|██▌       | 2/8 [00:27<01:22, 13.75s/it]


KeyboardInterrupt: 

Make video 

In [ ]:
import os
import cv2
from tqdm import tqdm

# folders
frame_folder       = 'rendered_frames'   # where your point images live, in subfolders 0,1,...,7
camera_pic_folder  = '../data'           # where your camera images live, same subfolder structure
output_folder      = 'videos'            # where the .mp4 files will be written

# make sure output_folder exists
os.makedirs(output_folder, exist_ok=True)

# parameters
fps = 30  # frames per second for the output video

def resize_to_height(img, target_h):
    h, w = img.shape[:2]
    scale = target_h / float(h)
    return cv2.resize(img, (int(w * scale), target_h))

for track_id in tqdm(range(8), desc="Making videos"):
    cam_dir   = os.path.join(camera_pic_folder,  str(track_id))
    point_dir = os.path.join(frame_folder,       str(track_id))
    
    # collect and sort PNGs by numeric name (e.g. "0.png", "1.png", ...)
    files = sorted(
        [f for f in os.listdir(cam_dir) if f.lower().endswith('.png')],
        key=lambda fn: int(os.path.splitext(fn)[0])
    )
    if not files:
        print(f" → no images in {cam_dir}, skipping")
        continue

    # load first pair to determine output size
    p0 = cv2.imread(os.path.join(point_dir, files[0]))
    c0 = cv2.imread(os.path.join(cam_dir,   files[0]))
    if p0 is None or c0 is None:
        raise RuntimeError(f"Could not read {files[0]} from one of the folders")

    # match heights
    H = max(p0.shape[0], c0.shape[0])
    p0 = resize_to_height(p0, H)
    c0 = resize_to_height(c0, H)

    # setup video writer
    W = p0.shape[1] + c0.shape[1]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_path = os.path.join(output_folder, f'track_{track_id}.mp4')
    writer = cv2.VideoWriter(out_path, fourcc, fps, (W, H))
    
    # write first frame
    writer.write(cv2.hconcat([p0, c0]))

    # process remaining frames
    for fname in files[1:]:
        p = cv2.imread(os.path.join(point_dir, fname))
        c = cv2.imread(os.path.join(cam_dir,   fname))
        if p is None or c is None:
            print(f"  ✗ skipping {fname} (failed to load)")
            continue
        p = resize_to_height(p, H)
        c = resize_to_height(c, H)
        writer.write(cv2.hconcat([p, c]))

    writer.release()
    print(f" → Saved video: {out_path}")


Making videos:  12%|█▎        | 1/8 [00:00<00:03,  1.97it/s]

 → Saved video: videos\track_0.mp4


Making videos:  25%|██▌       | 2/8 [00:03<00:11,  1.95s/it]

 → Saved video: videos\track_1.mp4


Making videos:  38%|███▊      | 3/8 [00:30<01:06, 13.28s/it]

...

 → Saved video: videos\track_5.mp4


Making videos:  88%|████████▊ | 7/8 [01:13<00:13, 13.63s/it]

 → Saved video: videos\track_6.mp4


Making videos: 100%|██████████| 8/8 [01:47<00:00, 13.41s/it]

 → Saved video: videos\track_7.mp4
